# Implanatação de algumas Bibliotacas

In [ ]:
import csv
import numpy as np
import pandas as pd
import sys
from tkinter import Tk
from tkinter import filedialog
from datetime import date, datetime, timedelta
#import os
#from ciso8601 import parse_datetime

# Criando variaveis para os arquivos permitidos

In [ ]:
p_cmpid3  = pd.read_excel('PQ378.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 378
param3    = pd.read_excel('LQ378.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 378

In [ ]:
control  = 0                                              ### controle de backup    
p_cmpid3['cmpid'] = p_cmpid3.cmpid.astype(str)            ### transforma o campo cmpid - QE 378 - do arq auxiliar para string

In [ ]:
## FUNÇÃO PARA SELEÇÃO DO ARQUIVO

class qe_s:

    def __init__(self):
        self.df = {nome[s]: [] for s in range(nome.shape[0])}
  
    def make_df(self):
        lines = []
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        for file in filedialog.askopenfilenames(filetypes=[("Arquivos de Text", "*.txt")]):
            for line in open(file, encoding="utf-8"):
                line = line.replace(",",".").strip()
                for x in range(nome.shape[0]):
                    self.df[nome[x]].append(line[i[x]:j[x]])
                    
        return pd.DataFrame(self.df)

In [ ]:
nome     = param3.Campo                      ### guarda os nomes das colunas existentes no quadro 376
i        = param3.Ind_inf                    ### guarda a posição no arquivo que inicia um determinado campo
j        = param3.Ind_sup                    ### guarda a posição no arquivo que termina um determinado campo

quadro = qe_s()       
df_main = quadro.make_df()
df_main

## SELECINE O(S) ARQUIVO(S) DESEJADO(S)

In [ ]:
###################################
###Importacao do Quadro378.xlsx ###
####que serão usados para fins#####
######### de comparação ###########
###################################

#Pasta CMPID
arq_cmpid = pd.read_excel('Quadro378.xlsx','CMPID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta RAMCODIGO
arq_ramcod = pd.read_excel('Quadro378.xlsx','RAMCODIGO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta ENTCODIGO
arq_entcod = pd.read_excel('Quadro378.xlsx','Cod_SUSEP',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MRFMESANO
arq_mrfmesano = pd.read_excel('Quadro378.xlsx','MRFMESANO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta rel_CMPID_TPMOID
arq_relacao = pd.read_excel('Quadro378.xlsx','rel_CMPID_TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMOID
arq_tpmoid = pd.read_excel('Quadro378.xlsx','TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta Bib_DefCamposEstatísticos
arq_defcamesta = pd.read_excel('Quadro378.xlsx','Bib_DefCamposEstatísticos',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

In [ ]:
# Criando uma lista para guardar todas as criticas impeditivas
criticas_impeditivas = []
criticas = []

# Criticas (Validacoes)

In [ ]:
#7394.3 Verifica se o campo sequencial ESPSEQ é uma sequência válida, que se inicia em 0000001
array3 = df_main['ESPSEQ']
x = list(array3)                                     ##Dados do arquivo
arrayteste = x[:]
arrayteste.sort()                                    ##Dados sequenciados
if (arrayteste == x):                                ##Comparando
    print('Tudo certo, esta de forma sequencial.')
else: 
    print('ERRO!!!, Não prossiga.')
    critica7394_3 = str('Critica 7394.3, Não temos uma sequencia valida na coluna ESRSEQ')
    criticas_impeditivas.append(critica7394_3)

In [ ]:
#7394.2 Verifica o tamanho padrão da linha (deve conter 173 caracteres)
tamanho = 0
for i in range(0,df_main.shape[0]):                    #Passando por cada linha
    for iten in df_main.columns:                       #Passando por cada coluna
        tamanho = tamanho + len(df_main[iten][i])      #Contando Caracteres por linha
    if tamanho != 173:
        print('Erro na linha ',i,' o número de caracteres é de: ',tamanho)
        critica7394_2 = ("Critica 7394.2, Erro na linha ",i," o número de caracteres é de: ",tamanho)
        criticas_impeditivas.append(critica7394_2)
    tamanho = 0

In [ ]:
#7394.4 Verifica se o campo ENTCODIGO corresponde à sociedade que está enviando o FIP/SUSEP
entcod = set(arq_entcod['Cod_SUSEP'].astype('str'))                   #Escolhendo a coluna do arquivo modelo
dadosentcod = set(df_main['ENTCODIGO'])                               #Escolhendo a coluna dos dados
for itemtentcod in dadosentcod: 
    if itemtentcod not in entcod:
        print('ERRO!! ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        critica7394_4 = ('Critica 7394.4, ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        criticas_impeditivas.append(critica7394_4)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtentcod)
        continue

In [ ]:
#7394.5 Verifica se o campo MRFMESANO corresponde, respectivamente, ao ano, mês e último dia do mês de referência do FIP/SUSEP 
df_main.loc[:,'DTMRFMESANO'] = df_main['MRFMESANO'].astype('datetime64')     #Criando uma nova coluna no formato Date
mrfmesano = set(arq_mrfmesano['DATAS'].astype('datetime64'))                 #Escolhendo a coluna do arquivo modelo
dadosmrfmesano = set(df_main['DTMRFMESANO'])                                 #Escolhendo a coluna dos dados
for itemmrfmesano in dadosmrfmesano: 
    if itemmrfmesano not in mrfmesano:
        print('ERRO!! MRFMESANO não corresponde a uma operação valida', itemmrfmesano)
        critica7394_5 = ('Critica 7394.5, MRFMESANO não corresponde a uma Data valida', itemmrfmesano)
        criticas_impeditivas.append(critica7394_5)
    else:
        print('MRFMESANO corresponde a uma Data válida.', itemmrfmesano)
        continue

In [ ]:
#7394.6 Verifica se o campo QUAID corresponde ao quadro 378 
array6 = set(df_main['QUAID'])           ##Verificando todos os Quadros que estamos tratando no documento.
for itemarray6 in array6:
    if itemarray6 == '378':
        print('Este se trata do Quadro:',itemarray6)
    else:
        print('ERRO!! Este se trata do Quadro:',itemarray6)
        critica7394_6 = ('Critica 7394.6, Este esta se tratando do(s) Quadro(s):',itemarray6)
        criticas_impeditivas.append(critica7394_6)

In [ ]:
#7394.7 Verifica se o campo TPMOID corresponde a um tipo de movimento válido (conforme tabela 'TiposMovimentos' do FIPSUSEP) 
tpmoid = set(arq_tpmoid['TPMOID'].astype('str'))                                    #Escolhendo a coluna do arquivo modelo
dadostpmoid = set(df_main['TPMOID'])                                                #Escolhendo a coluna dos dados
for itemtpmoid in dadostpmoid: 
    if itemtpmoid not in tpmoid:
        print('ERRO!! TPMOID não corresponde a uma operação valida', itemtpmoid)
        critica7394_7 = ('Critica 7394.7, TPMOID não corresponde a uma operação valida', itemtpmoid)
        criticas_impeditivas.append(critica7394_7)
    else:
        print('TPMOID corresponde a um tipo de operação válida.', itemtpmoid)
        continue

In [ ]:
#7394.8 Valida a correspondência entre os campos TPMOID e CMPID 
mod_relacao = set(arq_relacao['relacao'].astype('str'))
relacao = [df_main['CMPID'] + df_main['TPMOID']]
for itemrelacao in relacao:
    relacao2 = set(itemrelacao)
    for itemrelacao2 in relacao2:
        if itemrelacao2 not in mod_relacao:
            print('ERRO!! A relação ',itemrelacao2,' (CMPID/TPMOID) não é uma relação possivel.')
            critica7394_8 = ('Critica 7394.8, A relação ',itemrelacao2,' (CMPID/TPMOID) não é uma relação possivel.')
            criticas_impeditivas.append(critica7394_8)
        else:
            print('A relação ',itemrelacao2,' (CMPID/TPMOID) é uma relação possivel.')
        continue

In [ ]:
#7394.9 Verifica se o campo CMPID corresponde a um tipo de operação válida (conforme tabela 'Bib_DefCamposEstatísticos do FIPSUSEP) 
defcamesta = set(arq_defcamesta['CMPID'].astype('str'))             #Escolhendo a coluna do arquivo modelo
dadoscmpid = set(df_main['CMPID'])                                 #Escolhendo a coluna dos dados
for itemcmpid in dadoscmpid: 
    if itemcmpid not in defcamesta:
        print('ERRO!! O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        critica7394_7 = ('Critica 7394.7, O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        criticas.append(critica7394_7)
    else:
        print('O campo CMPID corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        continue

In [ ]:
#7394.10 Verifica se o campo RAMCODIGO corresponde, respectivamente, a um grupo de ramos e ramo válidos
ramcod = set(arq_ramcod['ramo_s1'].astype('str'))                       #Escolhendo a coluna do arquivo
dadosramcod = set(df_main['RAMCODIGO'])                                 #Escolhendo a coluna dos dados
for itemramcod in dadosramcod: 
    if itemramcod not in ramcod:
        print('ERRO!! RAMCODIGO não corresponde a uma operação valida', itemramcod)
        critica7394_10 = ('Critica 7394.10, RAMCODIGO não corresponde a uma operação valida', itemramcod)
        criticas.append(critica7394_10)
    else:
        print('RAMCODIGO corresponde a um tipo de operação válida.', itemramcod)
        continue

In [ ]:
#7394.11 Verifica se o campo RAMCODIGO não foi preenchido para os ramos 0588, 0589, 0983, 0986, 0991, 0992, 0994, 0996, 1066, 1383, 1386, 1391, 1392, 1396, 1603 e 2201 
ramcodver = set(arq_ramcod['RAMCODIGO_não_fazem_parte_376'].astype('str'))       #Escolhendo a coluna do arquivo modelo
dadosramcod = set(df_main['RAMCODIGO'])                                 #Escolhendo a coluna dos dados
for itemramcod in dadosramcod: 
    if itemramcod in ramcodver:
        print('ERRO!! RAMCODIGO não corresponde a uma operação valida', itemramcod)
        critica7394_11 = ('Critica 7394.11, RAMCODIGO não corresponde a uma operação valida', itemramcod)
        criticas.append(critica7394_11)
    else:
        print('RAMCODIGO corresponde a um tipo de operação válida.', itemramcod)
        continue

In [ ]:
#7394.13 Verifica se os campos ESPDATAINICIORO, ESPDATAFIMRO, ESPEMISSRO, ESPDATAINICIORD, ESPDATAFIMRD e ESPEMISSRD correspondem a uma data válida

#####################################################
##Readicionando as colunas de Data de str para date##
#####################################################

df_main.loc[:,'DTESPDATAINICIORO'] = df_main['ESPDATAINICIORO'].astype('datetime64')
df_main.loc[:,'DTESPDATAFIMRO'] = df_main['ESPDATAFIMRO'].astype('datetime64')
df_main.loc[:,'DTESDATAPEMISSRO'] = df_main['ESPDATAEMISSRO'].astype('datetime64')
df_main.loc[:,'DTESPDATAINICIORD'] = df_main['ESPDATAINICIORD'].astype('datetime64')
df_main.loc[:,'DTESPDATAFIMRD'] = df_main['ESPDATAFIMRD'].astype('datetime64')
df_main.loc[:,'DTESDATAPEMISSRD'] = df_main['ESPDATAEMISSRD'].astype('datetime64')

#Adicionado colunas ESPDATAINICIORO, ESPDATAFIMRO, ESPEMISSRO, ESPDATAINICIORD, ESPDATAFIMRD e ESPEMISSRD no tipo Date

In [ ]:
#7394.12 Verifica se o valor dos campos ESPVALORMOVRO, ESPVALORMOVRD, ESPVALORCARO, ESPVALORCARD, ESPVALORCIRO e ESPVALORCIRD é float 

###########################################
##Readicionando as colunas str para float##
###########################################

df_main.loc[:,'fESPVALORMOVRO'] = df_main['ESPVALORMOVRO'].astype('float64')
df_main.loc[:,'fESPVALORMOVRD'] = df_main['ESPVALORMOVRD'].astype('float64')
df_main.loc[:,'fESPVALORCARO'] = df_main['ESPVALORCARO'].astype('float64')
df_main.loc[:,'fESPVALORCARD'] = df_main['ESPVALORCARD'].astype('float64')
df_main.loc[:,'fESPVALORCIRO'] = df_main['ESPVALORCIRO'].astype('float64')
df_main.loc[:,'fESPVALORCIRD'] = df_main['ESPVALORCIRD'].astype('float64')

#Adicionado coluna ESPVALORMOVRO, ESPVALORMOVRD, ESPVALORCARO, ESPVALORCARD, ESPVALORCIRO e ESPVALORCIRD no tipo Float

In [ ]:
criticas_impeditivas  #Lista das criticas impeditivas

In [ ]:
df_criticas_i = pd.DataFrame(criticas_impeditivas)     #criacao do DataFrame das criticas impeditivas
df_criticas_i = df_criticas_i.rename(columns={0:'Descrição'})
df_criticas_i

In [ ]:
if criticas_impeditivas == []:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 378')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 378')
    df_criticas_i.to_csv('Criticas_Impeditivas_378.csv')   # Gerando csv das criticas impeditivas

# Criticas (Validacoes) Não Impeditivas

In [ ]:
# Criando DataFrame copia 

#Campos CMPID, ESPCODCESS, ENTCODIGO no formato int
df_mains = df_main.copy()
df_mains['CMPID'] = df_mains['CMPID'].astype('int64')
df_mains['ESPCODCESS'] = df_mains['ESPCODCESS'].astype('int64')
df_mains['ENTCODIGO'] = df_mains['ENTCODIGO'].astype('int64')

#Campo ESPFREQ no formato int
df_mains['ESPFREQ'] = df_mains['ESPFREQ'].astype('int64')

In [ ]:
#7394.14 Valida a correspondência entre os campos CMPID e ESPCODCESS
relacao15_26 = df_mains[(df_mains['CMPID'] == 1026) & (df_mains['ESPCODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao15_30 = df_mains[(df_mains['CMPID'] == 1030) & (df_mains['ESPCODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao15_34 = df_mains[(df_mains['CMPID'] == 1034) & (df_mains['ESPCODCESS'].values != df_mains['ENTCODIGO'].values)]

In [ ]:
i = 35   #Definindo numero de colunas

# Adicionando criticas com CMPIDs 1015, 1020, 1025 na lista 'criticas'
if relacao15_26.shape != (0,i): 
    critica7392_15_26 = ('Critica 7392.15_26, O CMPID é 1026 e o ESPCODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7392_15_26)
elif relacao15_30.shape != (0,i): 
    critica7392_15_30 = ('Critica 7392.15_30, O CMPID é 1030 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_30)
elif relacao15_34.shape != (0,i): 
    critica7392_15_34 = ('Critica 7392.15_34, O CMPID é 1034 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_34)
else: print('CMPIDs 1026, 1030, 1034. Valida campo ESPCODCESS')

In [ ]:
#7394.14 Valida a correspondência entre os campos CMPID e ESPCODCESS
relacao15_27 = df_mains[(df_mains['CMPID'] == 1027) & (df_mains['ESPCODCESS'].values <= 1) & (df_mains['ESPCODCESS'].values >= 9999)] 
relacao15_28 = df_mains[(df_mains['CMPID'] == 1028) & (df_mains['ESPCODCESS'].values <= 1) & (df_mains['ESPCODCESS'].values >= 9999)] 
relacao15_31 = df_mains[(df_mains['CMPID'] == 1031) & (df_mains['ESPCODCESS'].values <= 1) & (df_mains['ESPCODCESS'].values >= 9999)] 
relacao15_32 = df_mains[(df_mains['CMPID'] == 1032) & (df_mains['ESPCODCESS'].values <= 1) & (df_mains['ESPCODCESS'].values >= 9999)] 

In [ ]:
if relacao15_27.shape != (0,i): 
    critica7392_15_27 = ('Critica 7392.15_27, O CMPID é 1027 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_27)
elif relacao15_28.shape != (0,i): 
    critica7392_15_28 = ('Critica 7392.15_28, O CMPID é 1028 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_28)
elif relacao15_31.shape != (0,i): 
    critica7392_15_31 = ('Critica 7392.15_31, O CMPID é 1031 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_31)
elif relacao15_32.shape != (0,i): 
    critica7392_15_32 = ('Critica 7392.15_32, O CMPID é 1032 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_32)
else: print('CMPIDs 1027, 1028, 1031, 1032. Valida campo ESPCODCESS')

In [ ]:
#7394.14 Valida a correspondência entre os campos CMPID e ESPCODCESS
relacao15_29 = df_mains[(df_mains['CMPID'] == 1029) & (df_mains['ESPCODCESS'].values <= 30000) & (df_mains['ESPCODCESS'].values >= 59999)] 
relacao15_33 = df_mains[(df_mains['CMPID'] == 1033) & (df_mains['ESPCODCESS'].values <= 30000) & (df_mains['ESPCODCESS'].values >= 59999)] 
relacao15_37 = df_mains[(df_mains['CMPID'] == 1037) & (df_mains['ESPCODCESS'].values <= 30000) & (df_mains['ESPCODCESS'].values >= 59999)] 

In [ ]:
# Adicionando criticas com CMPIDs 1029, 1033, 1037 na lista 'criticas'
if relacao15_29.shape != (0,i): 
    critica7392_15_29 = ('Critica 7392.15_29, O CMPID é 1029 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_29)
elif relacao15_33.shape != (0,i): 
    critica7392_15_33 = ('Critica 7392.15_33, O CMPID é 1033 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_33)
elif relacao15_37.shape != (0,i): 
    critica7392_15_37 = ('Critica 7392.15_37, O CMPID é 1037 e o ESPCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_15_37)
else: print('CMPIDs 1029, 1033, 1037. Valida campo ESPCODCESS')

In [ ]:
#7394.15 Verifica se o campo ESPCODCESS corresponde a um código de sociedade válido 
dadosesrcodcess = set(df_main['ESPCODCESS'])       #Escolhendo a coluna dos dados
for itemtesrcodcess in dadosesrcodcess: 
    if itemtesrcodcess not in entcod:
        print('ERRO!! ESPCODCESS não corresponde a um código de sociedade válido', itemtesrcodcess)
        critica7394_15 = ('Critica 7394.15, ESPCODCESS corresponde a um código de sociedade válido', itemtesrcodcess)
        criticas.append(critica7394_15)
    else:
        print('ESPCODCESS corresponde a um tipo de operação válida.', itemtesrcodcess)
        continue

In [ ]:
#7394.16 Verifica se o campo ESPFREQ é um número inteiro positivo
dadosespfreq = set(df_mains['ESPFREQ'])
for itemespfreq in dadosespfreq:
    if itemespfreq <= 0:
        print('ERRO!! O campo ESPFREQ não corresponde um número inteiro positivo')
        critica7394_16 = ('Critica 7394.16, ESPFREQ não corresponde um número inteiro positivo', itemespfreq)
        criticas.append(critica7394_16)
    else:
        print('O campo ESPFREQ é um número inteiro positivo')

In [ ]:
#7394.18 Verifica se o ano da data dos campos ESPDATAINICIORO, ESPDATAFIMRO, ESPDATAEMISSRO, ESPDATAINICIORD, 
#ESPDATAFIMRD e ESPDATAEMISSRD está entre os limites de trinta anos para mais ou para menos do ano da data do 
#campo MRFMESANO 

#Criando um DataFrame com colunas de Data para cpmparação. 
#Adicionando MRFMESANO minimo e maximo
df_dates_comparacao = pd.DataFrame(df_main['DTMRFMESANO'] - timedelta(days = 10950))
df_dates_comparacao.loc[:,'DTMRFMESANO_max'] = df_main['DTMRFMESANO'] + timedelta(days = 10950)

In [ ]:
mask18_i = (df_main['DTESPDATAINICIORO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESPDATAINICIORO'] > df_dates_comparacao['DTMRFMESANO_max'])
df18_i = df_main.loc[mask18_i] 
if len(df18_i) > 0:
    df18_i.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18_i

In [ ]:
mask18_f = (df_main['DTESPDATAFIMRO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESPDATAFIMRO'] > df_dates_comparacao['DTMRFMESANO_max'])
df18_f = df_main.loc[mask18_f] 
if len(df18_f) > 0:
    df18_f.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18_f

In [ ]:
mask18_Eo = (df_main['DTESDATAPEMISSRO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESDATAPEMISSRO'] > df_dates_comparacao['DTMRFMESANO_max'])
df18_Eo = df_main.loc[mask18_Eo]
if len(df18_Eo) > 0:
    df18_Eo.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18_Eo

In [ ]:
mask18_Io = (df_main['DTESPDATAINICIORD'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESPDATAINICIORD'] > df_dates_comparacao['DTMRFMESANO_max'])
df18_Io = df_main.loc[mask18_Io]
if len(df18_Io) > 0:
    df18_Io.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18_Io

In [ ]:
mask18_Fo = (df_main['DTESPDATAFIMRD'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESPDATAFIMRD'] > df_dates_comparacao['DTMRFMESANO_max'])
df18_Fo = df_main.loc[mask18_Fo]
if len(df18_Fo) > 0:
    df18_Fo.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18_Fo

In [ ]:
mask18_e = (df_main['DTESDATAPEMISSRD'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESDATAPEMISSRD'] > df_dates_comparacao['DTMRFMESANO_max'])
df18_e = df_main.loc[mask18_e]
if len(df18_e) > 0:
    df18_e.loc[:,'ID da Critica'] = ('CRITICA 7394.18')
df18_e

In [ ]:
#7394.19 Verifica se a data do campo ESPDATAFIMRO é posterior à data do campo ESPDATAINICIORO e se a data do campo ESPDATAFIMRD é posterior à data do campo ESPDATAINICIORD
mask19 = (df_main['DTESPDATAFIMRO'] <= df_main['DTESPDATAINICIORO']) | (df_main['DTESPDATAFIMRD'] <= df_main['DTESPDATAINICIORD'])
df19 = df_main.loc[mask19]
if len(df19) > 0:
    df19.loc[:,'ID da Critica'] = ('CRITICA 7394.19')
df19

In [ ]:
#7394.20 Se o tipo de movimento for 'emissão', verifica se o mês e ano da data do campo ESPDATAEMISSRO 
#é igual ao mês e ano da data do campo MRFMESANO 
mask20_1 = (df_main['TPMOID'] == '0007')                   # Selecionando os movimento emissão

maskmr = df_main['DTMRFMESANO'].dt.strftime("%Y-%m")
masker = df_main['DTESDATAPEMISSRD'].dt.strftime("%Y-%m")
mask20_2 = (maskmr != masker)                              # Fazendo o comparativo dos campos ESPDATAEMISSRO e MRFMESANO

mask20 = (mask20_1) & (mask20_2)
df20 = df_main.loc[mask20]
if len(df20) > 0:
    df20.loc[:,'ID da Critica'] = ('CRITICA 7394.20')
df20

In [ ]:
#7394.21 Se o tipo de movimento for 'emissão', verifica se os valores dos campos ESPDATAINICIORO, 
#ESPDATAFIMRO, ESPDATAEMISSRO, ESPVALORMOVRO, ESPVALORCARO e ESPVALORCIRO são iguais, respectivamente, 
#aos valores dos campos ESPDATAINICIORD, ESPDATAFIMRD, ESPDATAEMISSRD, ESPVALORMOVRD, ESPVALORCARD e ESPVALORCIRD
mask21_1 = (df_main['TPMOID'] == '0007') & (df_main['ESPDATAINICIORO'] != df_main['ESPDATAINICIORD'])
df21_1 = df_main.loc[mask21_1]
if len(df21_1) > 0:
    df21_1.loc[:,'ID da Critica'] = ('CRITICA 7394.21')
df21_1

In [ ]:
mask21_2 = (df_main['TPMOID'] == '0007') & (df_main['ESPDATAFIMRO'] != df_main['ESPDATAFIMRD'])
df21_2 = df_main.loc[mask21_2]
if len(df21_2) > 0:
    df21_2.loc[:,'ID da Critica'] = ('CRITICA 7394.21')
df21_2

In [ ]:
mask21_3 = (df_main['TPMOID'] == '0007') & (df_main['ESPDATAEMISSRO'] != df_main['ESPDATAEMISSRD'])
df21_3 = df_main.loc[mask21_3]
if len(df21_3) > 0:
    df21_3.loc[:,'ID da Critica'] = ('CRITICA 7394.21')
df21_3

In [ ]:
mask21_4 = (df_main['TPMOID'] == '0007') & (df_main['ESPVALORMOVRO'] != df_main['ESPVALORMOVRD'])
df21_4 = df_main.loc[mask21_4]
if len(df21_4) > 0:
    df21_4.loc[:,'ID da Critica'] = ('CRITICA 7394.21')
df21_4

In [ ]:
mask21_5 = (df_main['TPMOID'] == '0007') & (df_main['ESPVALORCARO'] != df_main['ESPVALORCARD'])
df21_5 = df_main.loc[mask21_5]
if len(df21_5) > 0:
    df21_5.loc[:,'ID da Critica'] = ('CRITICA 7394.21')
df21_5

In [ ]:
mask21_6 = (df_main['TPMOID'] == '0007') & (df_main['ESPVALORCIRO'] != df_main['ESPVALORCIRD'])
df21_6 = df_main.loc[mask21_6]
if len(df21_6) > 0:
    df21_6.loc[:,'ID da Critica'] = ('CRITICA 7394.21')
df21_6

In [ ]:
#7394.22 Se o tipo de movimento for 'emissão' e o valor dos campos ESPVALORCARO e ESPVALORCIRO for 
#maior ou igual a zero, verifica se a soma dos campos ESPVALORCARO e ESPVALORCIRO é inferior ao valor 
#do campo ESPVALORMOVRO 
mask22_1 = (df_main['TPMOID'] == '0007') & (df_main['fESPVALORCARO'] >= 0) & (df_main['fESPVALORCIRO'] >= 0)
mask22_2 = df_main['fESPVALORCARO'] + df_main['fESPVALORCIRO']
mask22_3 = mask22_1 & (mask22_2 < df_main['fESPVALORCARO'])
df22 = df_main.loc[mask22_3]
if len(df22) > 0:
    df22.loc[:,'ID da Critica'] = ('CRITICA 7394.22')
df22

In [ ]:
#7394.23 Se o tipo de movimento for 'aumento' ou 'restituição' ou 'cancelamento'(0008,0009,0010), verifica se o mês e 
#ano da data do campo ESPDATAEMISSRD é igual ao mês e ano da data do campo MRFMESANO 
mask23_1 = (df_main['TPMOID'] == '0008') | (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010') # Selecionando os movimento emissão

maskmr = df_main['DTMRFMESANO'].dt.strftime("%Y-%m")
masker = df_main['DTESDATAPEMISSRD'].dt.strftime("%Y-%m")
mask23_2 = (maskmr != masker)                              # Fazendo o comparativo dos campos ESPDATAEMISSRO e MRFMESANO

mask23 = (mask23_1) & (mask23_2)
df23 = df_main.loc[mask23]
if len(df23) > 0:
    df23.loc[:,'ID da Critica'] = ('CRITICA 7394.23')
df23

In [ ]:
#7394.24 Se o tipo de movimento for 'aumento' ou 'restituição' ou 'cancelamento'(0008,0009,0010), verifica se a data 
#dos campos ESPDATAINICIORD e ESPDATAEMISSRD é igual ou posterior, respectivamente, à data dos campos 
#ESPDATAINICIORO e ESPDATAEMISSRO 
mask24_tpmoid = (df_main['TPMOID'] == '0008') | (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')

mask24_1 = mask24_tpmoid & (df_main['DTESPDATAINICIORD'] != df_main['DTESPDATAINICIORO'])
df24_1 = df_main.loc[mask24_1]
if len(df24_1) > 0:
    df24_1.loc[:,'ID da Critica'] = ('CRITICA 7394.24')
df24_1

In [ ]:
mask24_2 = mask24_tpmoid & (df_main['ESPDATAEMISSRD'] != df_main['ESPDATAEMISSRO'])
df24_2 = df_main.loc[mask24_2]
if len(df18_f) > 0:
    df24_2.loc[:,'ID da Critica'] = ('CRITICA 7394.24')
if len(df24_2) > 0:
    df24_2.loc[:,'ID da Critica'] = ('CRITICA 7394.24')
df24_2

In [ ]:
#7394.25  Se o tipo de movimento for 'aumento' ou 'restituição' ou 'cancelamento'(0008,0009,0010)e o valor dos campos ESPVALORCARD e 
#ESPVALORCIRD for maior que zero, verifica se a soma dos campos ESPVALORCARD e ESPVALORCIRD é inferior ao valor do campo ESPVALORMOVRD 
mask25_tpmoid = (df_main['TPMOID'] == '0008') | (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')
mask25_valor = (df_main['fESPVALORCARD'] > 0) & (df_main['fESPVALORCIRD'] > 0)
soma = (df_main['fESPVALORCARD']) + (df_main['fESPVALORCIRD'])

mask25 = (mask25_tpmoid) & (mask25_valor) & (soma >= df_main['fESPVALORMOVRD'])
df25 = df_main.loc[mask25]
if len(df25) > 0:
    df25.loc[:,'ID da Critica'] = ('CRITICA 7394.25')

df25

In [ ]:
#7394.26  Se o tipo de movimento for 'restituição' ou 'cancelamento', verifica se a data do campo 
#ESPDATAEMISSRD é inferior a 90 dias após à data informada no campo ESPDATAFIMRD

#Adicionando ESPDATAFIMRD + 90days
df_dates_comparacao.loc[:,'DTESPDATAFIMRD'] = df_main['DTESPDATAFIMRD'] + timedelta(days = 90)

mask26_tpmoid = (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')

mask26 = mask26_tpmoid & (df_main['DTESDATAPEMISSRD'] > df_dates_comparacao['DTESPDATAFIMRD'])
df26 = df_main.loc[mask26]
if len(df26) > 0:
    df26.loc[:,'ID da Critica'] = ('CRITICA 7394.26')

df26

In [ ]:
#7394.27 Se o tipo de movimento for 'restituição' ou 'cancelamento', verifica se a data do campo 
#ESPDATAFIMRD é igual à data do campo ESPDATAFIMRO 
mask27_tpmoid = (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')

mask27 = (mask27_tpmoid) & (df_main['DTESPDATAFIMRD'] != df_main['DTESPDATAFIMRO'])
df27 = df_main.loc[mask27]
if len(df27) > 0:
    df27.loc[:,'ID da Critica'] = ('CRITICA 7394.27')
df27

In [ ]:
#7394.28 Se o tipo de movimento for 'restituição' ou 'cancelamento', verifica se o valor dos campos ESPVALORMOVRD, ESPVALORCARD 
#e ESPVALORCIRD é igual ou inferior, respectivamente, ao valor dos campos ESPVALORMOVRO, ESPVALORCARO e ESPVALORCIRO
mask28_tpmoid = (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')

mask28 = (df_main['fESPVALORMOVRD'] > df_main['fESPVALORMOVRO']) | (df_main['fESPVALORCARD'] > df_main['fESPVALORCARO']) | (df_main['fESPVALORCIRD'] > df_main['fESPVALORCIRO'])
df28 = df_main.loc[mask28]
if len(df28) > 0:
    df28.loc[:,'ID da Critica'] = ('CRITICA 7394.28')
df28

In [ ]:
#7394.29 Para cada ramo, compara os valores registrados nos campos ESPVALORMOVRO, ESPVALORMOVRD, ESPVALORCIRO e ESPVALORCIRD com os valores informados no Quadro 02 do FIP/SUSEP (tolerância: R$ 1.000,00 ou 1% do valor informado no respectivo campo do Quadro 02, o que for maior) 

In [ ]:
#7394.30 Para cada ramo, compara os valores registrados nos campos ESPVALORCARO e ESPVALORCARD com os valores informados no Quadro 08 do FIP/SUSEP (tolerância: R$ 1.000,00 ou 1% do valor informado no respectivo campo do Quadro 08, o que for maior)

In [ ]:
#7394.31 Verifica se o valor do campo ESPVALORMOVRO é maior do que zero e se o valor dos campos 
#ESPVALORCARO, ESPVALORCARD, ESPVALORCIRO e ESPVALORCIRD é maior ou igual a zero
mask31 = df_main['fESPVALORMOVRO']
mask31_1 = (mask31 > 0) & (df_main['fESPVALORMOVRO'] < 0) | (df_main['fESPVALORCARD'] < 0) | (df_main['fESPVALORCIRO'] < 0) | (df_main['fESPVALORCIRD'] < 0)
df31 = df_main.loc[mask31_1]
if len(df31) > 0:
    df31.loc[:,'ID da Critica'] = ('CRITICA 7394.31')
df31

In [ ]:
#7394.32 Se o tipo de operação for 'cosseguro aceito' ou 'cosseguro cedido' ou 'prêmio de resseguro' ou 'restituição de prêmio
#de cosseguro aceito' ou 'restituição de prêmio de cosseguro cedido' ou 'restituição de prêmio de resseguro' ou
#'cancelamento total de prêmio de cosseguro aceito' ou 'cancelamento total de prêmio de cosseguro cedido' ou
#'cancelamento total de prêmio de resseguro', verifica se o valor dos campos ESPVALORCIRO e ESPVALORCIRD é igual a zero
# CMPID 1027, 1028, 1029,1031, 1032, 1033, 1035, 1036, 1037
mask_cmpid_1 = (df_main['CMPID'] == '1027') | (df_main['CMPID'] == '1028') | (df_main['CMPID'] == '1029')
mask_cmpid_2 = (df_main['CMPID'] == '1031') | (df_main['CMPID'] == '1032') | (df_main['CMPID'] == '1033')
mask_cmpid_3 = (df_main['CMPID'] == '1035') | (df_main['CMPID'] == '1036') | (df_main['CMPID'] == '1037')
mask32_cmpid = (mask_cmpid_1) | (mask_cmpid_2) | (mask_cmpid_3)
mask32_valor = (df_main['fESPVALORCIRO'] != 0.0) | (df_main['fESPVALORCIRD'] != 0.0)

mask32 = (mask32_cmpid) & (mask32_valor)
df32 = df_main.loc[mask32]
if len(df32) > 0:
    df32.loc[:,'ID da Critica'] = ('CRITICA 7394.32')
df32

In [ ]:
#7394.33 Se o tipo de operação for 'prêmio de resseguro' ou 'restituição de prêmio de resseguro' ou 
#'cancelamento de prêmio de resseguro', verifica se o valor dos campos ESPVALORCARO e ESPVALORCARD é igual a zero
# CMPID 1029, 1033, 1037
mask33_cmpid = (df_main['CMPID'] == '1029') | (df_main['CMPID'] == '1033') | (df_main['CMPID'] == '1037')

mask33 = (mask33_cmpid) & (df_main['fESPVALORCARO'] != 0.0) & (df_main['fESPVALORCARD'] != 0.0)
df33 = df_main.loc[mask33]
if len(df33) > 0:
    df33.loc[:,'ID da Critica'] = ('CRITICA 7394.33')
df33

In [ ]:
#7394.34 Se o tipo de movimento for 'emissão', verifica se o campo RAMCODIGO não se refere a um ramo em run-off 
#(conforme Anexo II da Circular SUSEP nº 395/2009)
dadosrunoff = set(arq_ramcod['run-off'].astype('str'))          # Escolhendo a coluna do arquivo
mask_tpmoid = (df_main['TPMOID'] == '0007')                     # Criando uma mascara com os TPMOID emissão
df34 = df_main.loc[mask_tpmoid]                                 # Transformando a mascara em DataFrame
dadosramcodoff = set(df34['RAMCODIGO'])                         # Escolhendo a coluna do DataFrame df34
for itemramcodoff in dadosramcodoff: 
    if itemramcodoff in dadosrunoff:
        print('ERRO!! O campo RAMCODIGO se refere a um ramo em run-off', itemramcodoff)
        critica7394_34 = ('Critica 7394.34, O campo RAMCODIGO se refere a um ramo em run-off', itemramcodoff)
        criticas.append(critica7394_34)
    else:
        print('O campo RAMCODIGO não se refere a um ramo em run-off', itemramcodoff)
        continue

In [ ]:
#7394.35 Para os tipos de movimento “Emissão de Prêmio”, “Aumento de Prêmio” e “Restituição de Prêmio 
#(Cancelamento Parcial de Prêmio)”,(0007,0008,0009) verifica se o valor do campo ESPVALORMOVRD é maior do que zero
mask_tpmoid = (df_main['TPMOID'] == '0007') | (df_main['TPMOID'] == '0008') | (df_main['TPMOID'] == '0009')

mask_35 = (mask_tpmoid) & (df_main['fESPVALORMOVRD'] <= 0)
df35 = df_main.loc[mask_35]
if len(df35) > 0:
    df35.loc[:,'ID da Critica'] = ('CRITICA 7394.35')
df35

In [ ]:
#7394.36 Se o MRFMESANO for maior ou igual a 20180131 e o tipo de movimento for “Aumento de Prêmio”, 
#verificar se o campo ESPVALORCIRD é igual a zero
mask_36 = (df_main['MRFMESANO'] >= '20180131') & (df_main['TPMOID'] >= '0008') & (df_main['fESPVALORCIRD'] != 0)
df36 = df_main.loc[mask_36]
if len(df36) > 0:
    df36.loc[:,'ID da Critica'] = ('CRITICA 7394.36')
df36

In [ ]:
#7394.37 Se o MRFMESANO for maior ou igual a 20180131 e o tipo de movimento for “Emissão de Prêmio”, 
#verificar se o campo ESPVALORCIRO é igual a zero 
mask_37 = (df_main['MRFMESANO'] >= '20180131') & (df_main['TPMOID'] >= '0007') & (df_main['fESPVALORCIRO'] != 0)
df37 = df_main.loc[mask_37]
if len(df37) > 0:
    df37.loc[:,'ID da Critica'] = ('CRITICA 7394.37')
df37

In [ ]:
################################################
# Adicionando Nome do Criador do output e data #
################################################

df_main.loc[:,'Criador'] = input("Insira seu nome: ") ### insere a coluna com o nome do criador da tabela
data_presente = datetime.now()
df_main.loc[:,'Data do output'] = data_presente ### insere a coluna com a data de criacao da tabela

In [ ]:
df_main

In [ ]:
criticas

In [ ]:
df_criticas = pd.DataFrame(criticas)
df_criticas = df_criticas.rename(columns={0:'Descrição'})
for i in range(0,df_criticas.shape[0]):
    df_criticas.loc[:,'Quantidade de Criticas'] = i
df_criticas

In [ ]:
df_criticas_2 = pd.concat([df18_i,df18_f,df18_Eo,df18_Io,df18_Fo,df18_e,df19,df20,df21_1,df21_2,df21_3,df21_4,df21_5,df21_6,df22,df23,df24_1,df24_2,df25,df26,df27,df28,df31,df32,df33,df35,df36,df37])
df_criticas_2

In [ ]:
if criticas == []:                   # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 378')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 378')
    df_criticas.to_csv('Criticas_nao_impeditivas378.csv')     # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 378')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 378')
    df_criticas_2.to_csv('Criticas_nao_impeditivas378_2.csv')         # Gerando csv das criticas impeditivas

In [ ]:
df_main.to_csv('378_Trabalho.csv')

# Teste

In [ ]:
df_main['mask'] = pd.to_datetime(df_main['DTESDATAPEMISSRD'])
#mask = pd.to_datetime(mask04)
df_main['mask'] = df_main['mask'].dt.strftime("%d/%m/%Y")
df_main['mask']